# Analysis code for the "First, you need a Gestalt: an interaction of bottom-up and top-down streams during perception of an ambiguously rotating human walker" maniscript.
Within group analysis using linear mixed models. Generates __"Within"__ subtable in __Figure 1A__.

In [7]:
rm(list=ls())

In [8]:
library(dplyr)
library(ggplot2)
library(tidyr)
library(lmerTest)
library(lmPerm)

Loading individual observers into a single table

In [9]:
groups <- c('Naïve', 'Informed')
results <- data.frame()

for(current.group in groups){
    group.files <- list.files(path= current.group, pattern='csv$')
    for(current.file in group.files){
        current.observer <- read.csv2(file.path(current.group, current.file), dec = '.')
        
        # adding group ID
        current.observer$Group <- current.group
        
        # merging
        results <- rbind(results, current.observer) 
    }
}

# Block 0 was training only, excluding it from the analysis
results <- subset(results, Block>0)

# Putting factors into a proper order
results$Condition <- factor(results$Condition, levels= c('Scrambled static', 'Upright walker', 'Inverted walker', 'Scrambled moving'))
results$Group <- factor(results$Group, levels= c('Naïve', 'Informed'))

#### Computing the fraction of the time that observers reported perception of illusory rotation

In [10]:
results$Observer <- as.character(results$Observer)

# splitting into groups to simplify the completion
naive.rotation <- results %>%
    dplyr::filter(Group=='Naïve') %>%
    dplyr::group_by(Group, Condition, Observer) %>%
    dplyr::summarize(visible.fraction= 100*sum(Duration)/Block.duration[1]) %>%
    dplyr::mutate(Observer= as.factor(Observer)) %>%
    tidyr::complete(Observer,  fill = list(visible.fraction = 0)) %>%
    dplyr::mutate(Observer= as.character(Observer))
informed.rotation <- results %>%
    dplyr::filter(Group=='Informed') %>%
    dplyr::group_by(Group, Condition, Observer) %>%
    dplyr::summarize(visible.fraction= 100*sum(Duration)/Block.duration[1]) %>%
    dplyr::mutate(Observer= as.factor(Observer)) %>%
    tidyr::complete(Observer,  fill = list(visible.fraction = 0)) %>%
    dplyr::mutate(Observer= as.character(Observer))

rotation <- rbind(naive.rotation, informed.rotation)
results$Observer <- as.factor(results$Observer)

In [11]:
rcontrast<-function(t, df) {
    return(sqrt(t^2/(t^2 + df)))
}

naive.rotation <- subset(rotation, Group=='Naïve')
naive.summary<-data.frame(summary(lmerTest::lmer(visible.fraction ~ Condition + (1 | Observer), data= naive.rotation))$coefficients)
naive.summary$effect.size <- rcontrast(naive.summary$t.value, naive.summary$df)
naive.summary
summary(lmerTest::lmer(visible.fraction ~ Condition + (1 | Observer), data= naive.rotation))

,Estimate,Std..Error,df,t.value,Pr...t..,effect.size
(Intercept),91.06337,5.133823,68.00027,17.737925,0.000000e+00,0.9067987
ConditionUpright walker,-22.73340,7.260322,68.00027,-3.131184,2.566565e-03,0.3549817
ConditionInverted walker,-80.93328,7.260322,68.00027,-11.147340,0.000000e+00,0.8039386
ConditionScrambled moving,-53.91463,7.260322,68.00027,-7.425928,2.412079e-10,0.6691799


Linear mixed model fit by REML t-tests use Satterthwaite approximations to
  degrees of freedom [lmerMod]
Formula: visible.fraction ~ Condition + (1 | Observer)
   Data: naive.rotation

REML criterion at convergence: 623.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1371 -0.4651  0.1065  0.4448  2.3929 

Random effects:
 Groups   Name        Variance Std.Dev.
 Observer (Intercept)   0.0     0.00   
 Residual             474.4    21.78   
Number of obs: 72, groups:  Observer, 18

Fixed effects:
                          Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)                 91.063      5.134  68.000  17.738  < 2e-16 ***
ConditionUpright walker    -22.733      7.260  68.000  -3.131  0.00257 ** 
ConditionInverted walker   -80.933      7.260  68.000 -11.147  < 2e-16 ***
ConditionScrambled moving  -53.915      7.260  68.000  -7.426 2.41e-10 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
        

In [12]:
informed.rotation <- subset(rotation, Group=='Informed')
informed.summary<-data.frame(summary(lmerTest::lmer(visible.fraction ~ Condition + (1 | Observer), data= informed.rotation))$coefficients)
informed.summary$effect.size <- rcontrast(informed.summary$t.value, informed.summary$df)
informed.summary
summary(lmerTest::lmer(visible.fraction ~ Condition + (1 | Observer), data= informed.rotation))

,Estimate,Std..Error,df,t.value,Pr...t..,effect.size
(Intercept),89.875685,5.825483,62.39223,15.4280222,0.000000e+00,0.89012020
ConditionUpright walker,2.857469,7.491653,50.99947,0.3814203,7.044759e-01,0.05333379
ConditionInverted walker,-22.420398,7.491653,50.99947,-2.9927170,4.254336e-03,0.38650041
ConditionScrambled moving,-47.351305,7.491653,50.99947,-6.3205415,6.433318e-08,0.66275957


Linear mixed model fit by REML t-tests use Satterthwaite approximations to
  degrees of freedom [lmerMod]
Formula: visible.fraction ~ Condition + (1 | Observer)
   Data: informed.rotation

REML criterion at convergence: 638.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7984 -0.3888  0.1067  0.6670  1.6562 

Random effects:
 Groups   Name        Variance Std.Dev.
 Observer (Intercept) 105.7    10.28   
 Residual             505.1    22.47   
Number of obs: 72, groups:  Observer, 18

Fixed effects:
                          Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)                 89.876      5.825  62.390  15.428  < 2e-16 ***
ConditionUpright walker      2.857      7.492  51.000   0.381  0.70448    
ConditionInverted walker   -22.420      7.492  51.000  -2.993  0.00425 ** 
ConditionScrambled moving  -47.351      7.492  51.000  -6.321 6.43e-08 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
     

In [16]:
720/60

[1] 12